# Build Sample

After running *download_text_files.ipynb*, we can now build the sample of data to use in the analysis.  The main output of this file is *output/analysis_sample.csv* which includes the following:
1. 5 positive sentiment scores
1. 5 negative sentiment scores
1. a return variable from the day of the 10-K's release to two days following
1. a return variable from two days after the 10-K's release to ten days following 
1. the 10-K's word count
1. the 10-K's unique word count
1. accounting variables from the CCM

## Setup

In [1]:
# File and text handling
import glob
import os
import re
import pandas as pd
from bs4 import BeautifulSoup
from near_regex import NEAR_regex
from tqdm import tqdm

# Gathering 2022 returns
from zipfile import ZipFile
from urllib.request import urlopen
from io import BytesIO

# Get filing dates from SEC EDGAR
from requests_html import HTMLSession
from time import sleep

In [2]:
# Paths and file handling
input_dir = 'inputs'
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Inputs
topic_path = input_dir + '/topic_list.csv'
sp500_path = input_dir + '/s&p500_2022.csv'
firm_10k_path = '10k_files/sec-edgar-filings'
firm_10k_clean_path = '10k_files/clean'

# Outputs
sentiment_save_path = output_dir + '/ticker_sentiments.csv'    # store intermediate results
returns_save_path = output_dir + '/ticker_returns.csv'         # store intermediate results
final_save_path = output_dir + '/analysis_sample.csv'

In [3]:
# Load S&P500 companies into a dataframe
try:
    sp500_orig = pd.read_csv(sp500_path)[['Symbol', 'Security', 'CIK', 'truth_path']]
except Exception as error:
    print('Please run the contents of download_text_files.ipynb before proceeding')
    print(repr(error))
    
sp500_orig

,Symbol,Security,CIK,truth_path
0,MMM,3M,66740,MMM
1,AOS,A. O. Smith,91142,AOS
2,ABT,Abbott,1800,ABT
3,ABBV,AbbVie,1551152,ABBV
4,ACN,Accenture,1467373,ACN
...,...,...,...,...
498,YUM,Yum! Brands,1041061,YUM
499,ZBRA,Zebra Technologies,877212,ZBRA
500,ZBH,Zimmer Biomet,1136869,ZBH
501,ZION,Zions Bancorporation,109380,0000109380


## Load 2022 returns

Because this step is not costly compared to others, we gather the filing dates regardless of whether we have done so before.

Once we have the filing dates, we extract only the 11 days following the announcement.  We then can compute returns for each period of days.

In [4]:
# Store the 2022 returns in a separate table
sp500_rets = sp500_orig.copy()

# Get the filing date for each 10-K
session = HTMLSession()

for i in tqdm(range(len(sp500_rets))):
    tic = sp500_rets['Symbol'].iloc[i]
    cik = sp500_rets['CIK'].iloc[i]
    truth_path = sp500_rets['truth_path'].iloc[i]
    
    if not os.path.exists(fr'{firm_10k_path}/{truth_path}/10-K/'):
        print(f'Error finding accession number for ticker {tic}, cik {cik}')
        continue
    accession = os.listdir(fr'{firm_10k_path}/{truth_path}/10-K/')[0]
    
    url = f'https://www.sec.gov/Archives/edgar/data/{cik}/{accession}-index.html'
    r = session.get(url)
    try:
        sp500_rets.loc[i, 'filing_date'] = r.html.find('.info', first=True).text
    except Exception as error:
        print(f'Could not get filing date for ticker {tic}, cik {cik}, accession number {accession}: {repr(error)}')
    sleep(0.1)

sp500_rets

 39%|███████████████████████████████▎                                                | 197/503 [00:52<01:18,  3.91it/s]

Error finding accession number for ticker FRC, cik 1132979


 42%|█████████████████████████████████▉                                              | 213/503 [00:55<01:19,  3.64it/s]

Error finding accession number for ticker GEHC, cik 1932393


 83%|██████████████████████████████████████████████████████████████████▎             | 417/503 [01:45<00:30,  2.86it/s]

Error finding accession number for ticker SBNY, cik 1288784


100%|████████████████████████████████████████████████████████████████████████████████| 503/503 [02:05<00:00,  4.01it/s]


,Symbol,Security,CIK,truth_path,filing_date
0,MMM,3M,66740,MMM,2022-02-09
1,AOS,A. O. Smith,91142,AOS,2022-02-11
2,ABT,Abbott,1800,ABT,2022-02-18
3,ABBV,AbbVie,1551152,ABBV,2022-02-18
4,ACN,Accenture,1467373,ACN,2022-10-12
...,...,...,...,...,...
498,YUM,Yum! Brands,1041061,YUM,2022-02-28
499,ZBRA,Zebra Technologies,877212,ZBRA,2022-02-10
500,ZBH,Zimmer Biomet,1136869,ZBH,2022-02-25
501,ZION,Zions Bancorporation,109380,0000109380,2022-02-25


In [5]:
# Download 2022 CSRP returns
url = "https://github.com/LeDataSciFi/data/raw/main/Stock%20Returns%20(CRSP)/crsp_2022_only.zip"
with urlopen(url) as request:
    data = BytesIO(request.read())

with ZipFile(data) as archive:
    with archive.open(archive.namelist()[0]) as stata:
        stock_rets = pd.read_stata(stata)

stock_rets

,ticker,date,ret
0,JJSF,2021-12-01,-0.011276
1,JJSF,2021-12-02,0.030954
2,JJSF,2021-12-03,0.000287
3,JJSF,2021-12-06,0.014362
4,JJSF,2021-12-07,0.012459
...,...,...,...
2587061,TSLA,2022-12-23,-0.017551
2587062,TSLA,2022-12-27,-0.114089
2587063,TSLA,2022-12-28,0.033089
2587064,TSLA,2022-12-29,0.080827


In [6]:
# Based on filing date, add return from t to t+2 and from t+3 to t+10
combined_rets = sp500_rets.merge(
        stock_rets.rename(columns={'ticker':'Symbol'}),
        on='Symbol',
        how='left',
        validate='1:m')
combined_rets = combined_rets.query('filing_date <= date') \
        .sort_values(by='date') \
        .groupby('Symbol') \
        .head(11)
combined_rets['agg_ret'] = 1 + combined_rets['ret']

combined_rets['ret_t-t2'] = combined_rets.groupby('Symbol') \
        .head(3)['agg_ret'] \
        .cumprod() - 1
combined_rets['ret_t3-t10'] = combined_rets.groupby('Symbol') \
        .tail(8)['agg_ret'] \
        .cumprod() - 1
final_rets = combined_rets.groupby('Symbol') \
        .head(3).groupby('Symbol') \
        .tail(1)[['Symbol', 'Security', 'CIK', 'filing_date', 'ret_t-t2']]
final_rets = final_rets.merge(
        combined_rets.groupby('Symbol') \
                .tail(1)[['Symbol', 'Security', 'CIK', 'ret_t3-t10']],
        on=['Symbol', 'Security', 'CIK'],
        validate='1:1',
        how='left')

final_rets.to_csv(returns_save_path, index=False)
final_rets

,Symbol,Security,CIK,filing_date,ret_t-t2,ret_t3-t10
0,ADBE,Adobe Inc.,796343,2022-01-21,-0.015915,0.004955
1,LMT,Lockheed Martin,936468,2022-01-25,-0.034930,0.057001
2,SLB,Schlumberger,87347,2022-01-26,0.066684,0.970083
3,URI,United Rentals,1067701,2022-01-26,0.072747,0.202582
4,INTC,Intel,50863,2022-01-27,0.165922,0.660988
...,...,...,...,...,...,...
492,KEYS,Keysight,1601046,2022-12-15,0.864134,-0.993890
493,AVGO,Broadcom Inc.,1730168,2022-12-16,0.827125,-0.993797
494,AMAT,Applied Materials,6951,2022-12-16,0.818515,-0.993813
495,NDSN,Nordson Corporation,72331,2022-12-19,0.888110,-0.993850


## Clean 10-Ks

For each 10-K, we remove xml and html tags, extracting only the text.  We make the entire document lowercase and keep only words. If this step has already been done, we skip it.

In [7]:
os.makedirs(firm_10k_clean_path, exist_ok=True)
sp500_sents = sp500_orig.copy()

for i in tqdm(range(len(sp500_sents))):
    tic = sp500_sents['Symbol'].iloc[i]
    truth_path = sp500_sents['truth_path'].iloc[i]    # Locate the 10-K

    # Check existence of path
    if not os.path.exists(fr'{firm_10k_path}/{truth_path}'):
        print(f'Cannot find 10-K for ticker {tic}')
        continue
    
    # Create clean path
    os.makedirs(fr'{firm_10k_clean_path}/{truth_path}', exist_ok=True)
    
    for path in glob.glob(fr'{firm_10k_path}/{truth_path}/*/*/*.html'):
        # Check existence of cleaned 10-K
        if os.path.exists(fr'{firm_10k_clean_path}/{truth_path}/10-K.txt'):
            continue
        
        # Open and clean the 10-K
        with open(path, 'rb') as report_file:
            html = report_file.read()
        soup = BeautifulSoup(html, 'lxml-xml')
        for div in soup.find_all("div", {'style': 'display:none'}):
            div.dintlpose()                       # remove hidden divs,
        lower = soup.get_text().lower()           # uppercase,
        no_punc = re.sub(r'\W', ' ', lower)       # non-alpha-numeric,
        cleaned = re.sub(r'\s+', ' ', no_punc)    # single-space
        
        # Persist changes to the clean directory
        result_path = fr'{firm_10k_clean_path}/{truth_path}/10-K.txt' 
        with open(result_path, 'wb') as result_file:
            result_file.write(cleaned.encode('utf-8'))

 65%|███████████████████████████████████████████████████▏                           | 326/503 [00:00<00:00, 647.36it/s]

Cannot find 10-K for ticker FRC
Cannot find 10-K for ticker GEHC


100%|███████████████████████████████████████████████████████████████████████████████| 503/503 [00:00<00:00, 615.64it/s]

Cannot find 10-K for ticker SBNY


## Load sentiment dictionaries

Now, we load the sentiment dictionaries.

For the LM dictionary, we only keep positive values.  The dictionary was released in 2021, and the year that any word was added to the list is included as a positive value.  The year a word was removed from the list, it is included as a negative value.  Any negative values have been removed from the list during some release, so we should only keep the positive values.

In [8]:
# ML Dictionaries
with open('inputs/ML_negative_unigram.txt', 'r') as file:
    BHR_negative = [line.strip() for line in file]
with open('inputs/ML_positive_unigram.txt', 'r') as file:
    BHR_positive = [line.strip() for line in file]

In [9]:
# LM Dictionaries
LM = pd.read_csv('inputs/LM_MasterDictionary_1993-2021.csv')
LM_negative = LM.query('Negative > 0')['Word'].to_list()
LM_positive = LM.query('Positive > 0')['Word'].to_list()

## Generate sentiment regex

We start by gathering the sentiments into regex patterns.  We define a function to generate the NEAR_regex for each pair of two topics, and 

In [10]:
# Gather sentiments into regex
BHR_negative_regex = '(' + '|'.join(BHR_negative).lower() + ')'
BHR_positive_regex = '(' + '|'.join(BHR_positive).lower() + ')'
LM_negative_regex = '(' + '|'.join(LM_negative).lower() + ')'
LM_positive_regex = '(' + '|'.join(LM_positive).lower() + ')'

In [11]:
# Topic regex
def NEAR_regex_helper(topic_df, topic_name, max_words_between=5):
    topic_list = topic_df['term'].loc[topic_df['type'] == topic_name + '_topic']
    positive_list = topic_df['term'].loc[topic_df['type'] == topic_name + '_positive']
    negative_list = topic_df['term'].loc[topic_df['type'] == topic_name + '_negative']
    
    topic_regex = '(' + '|'.join(topic_list).lower() + ')'
    positive_regex = '(' + '|'.join(positive_list).lower() + ')'
    negative_regex = '(' + '|'.join(negative_list).lower() + ')'
    
    positive_regex = NEAR_regex([topic_regex, positive_regex], max_words_between=max_words_between)
    negative_regex = NEAR_regex([topic_regex, negative_regex], max_words_between=max_words_between)
    return [positive_regex, negative_regex]

# Read in file
if not os.path.exists(topic_path):
    print(f'Cannot find path {topic_path} to topic list')
else:
    topic_df = pd.read_csv(topic_path)
    
    # Get topics from list
    fs_positive_regex, fs_negative_regex = NEAR_regex_helper(topic_df, 'fs')
    cg_positive_regex, cg_negative_regex = NEAR_regex_helper(topic_df, 'cg')
    ecom_positive_regex, ecom_negative_regex = NEAR_regex_helper(topic_df, 'ecom')
    
    ''' Other existing topics
    esg_positive_regex, esg_negative_regex = NEAR_regex_helper(topic_df, 'esg')
    intl_positive_regex, intl_negative_regex = NEAR_regex_helper(topic_df, 'intl')
    bio_positive_regex, bio_negative_regex = NEAR_regex_helper(topic_df, 'bio')
    cs_positive_regex, cs_negative_regex = NEAR_regex_helper(topic_df, 'cs')
    csat_positive_regex, csat_negative_regex = NEAR_regex_helper(topic_df, 'csat')
    esat_positive_regex, esat_negative_regex = NEAR_regex_helper(topic_df, 'esat')
    fp_positive_regex, fp_negative_regex = NEAR_regex_helper(topic_df, 'fp')
    fs_positive_regex, fs_negative_regex = NEAR_regex_helper(topic_df, 'fs')
    ia_positive_regex, ia_negative_regex = NEAR_regex_helper(topic_df, 'ia')
    re_positive_regex, re_negative_regex = NEAR_regex_helper(topic_df, 're')
    sc_positive_regex, sc_negative_regex = NEAR_regex_helper(topic_df, 'sc')
    tech_positive_regex, tech_negative_regex = NEAR_regex_helper(topic_df, 'tech')
    '''

## Load each firm and add sentiment variables

Now, we add the sentiment measures for each 10-K.

In [12]:
# Simplify adding a value to a given row
def add_sentiment(df, i, sentiment_name, search, text, word_count):
    df.loc[i, sentiment_name] = len(re.findall(search, text)) / word_count

In [13]:
# Load and get the sentiment of clean 10-Ks
temp_path = output_dir + '/ticker_sentiments_temp.csv'

# Compute sentiment ariables
if not os.path.exists(sentiment_save_path):
    for i in tqdm(range(len(sp500_sents))):
        tic = sp500_sents['Symbol'].iloc[i]
        truth_path = sp500_sents['truth_path'].iloc[i]    # Get the path where the clean 10-K is stored

        # Check existence of path
        if not os.path.exists(fr'{firm_10k_clean_path}/{truth_path}'):
            print(f'Cannot find clean 10-K for ticker {tic}')
            continue

        for path in glob.glob(fr'{firm_10k_clean_path}/{truth_path}/*.txt'):
            with open(path, 'rb') as report_file:
                cleaned = str(report_file.read())

            # Add word count and unique word count
            word_list = re.findall(r'\w+', cleaned)
            sp500_sents.loc[i, 'unique_word_count'] = len(set(word_list))
            word_count = len(word_list)
            sp500_sents.loc[i, 'word_count'] = word_count

            # Gather valence variables
            add_sentiment(sp500_sents, i, 'bhr_negative', BHR_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'bhr_positive', BHR_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'lm_negative', LM_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'lm_positive', LM_positive_regex, cleaned, word_count)

            # Gather topic valence variables
            add_sentiment(sp500_sents, i, 'fs_negative', fs_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'fs_positive', fs_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'cg_negative', cg_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'cg_positive', cg_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'ecom_negative', ecom_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'ecom_positive', ecom_positive_regex, cleaned, word_count)
            
            ''' Other existing topics
            add_sentiment(sp500_sents, i, 'esg_negative', esg_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'esg_positive', esg_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'intl_negative', intl_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'intl_positive', intl_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'bio_negative', bio_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'bio_positive', bio_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'cs_negative', cs_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'cs_positive', cs_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'csat_negative', csat_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'csat_positive', csat_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'esat_negative', esat_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'esat_positive', esat_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'fp_negative', fp_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'fp_positive', fp_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'fs_negative', fs_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'fs_positive', fs_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'ia_negative', ia_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'ia_positive', ia_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 're_negative', re_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 're_positive', re_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'sc_negative', sc_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'sc_positive', sc_positive_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'tech_negative', tech_negative_regex, cleaned, word_count)
            add_sentiment(sp500_sents, i, 'tech_positive', tech_positive_regex, cleaned, word_count)
            '''

            # Save intermittently
            if i % 50 == 0:
                sp500_sents.to_csv(temp_path, index=False)

    sp500_sents.to_csv(sentiment_save_path, index=False)
else:
    # Load existing sentiments
    sp500_sents = pd.read_csv(sentiment_save_path)
sp500_sents.drop(['CIK'], axis=1).describe()

,unique_word_count,word_count,bhr_negative,bhr_positive,lm_negative,lm_positive,esg_negative,esg_positive,intl_negative,intl_positive,...,fp_negative,fp_positive,ia_negative,ia_positive,re_negative,re_positive,sc_negative,sc_positive,tech_negative,tech_positive
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,...,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,5425.424000,69504.586000,0.038030,0.039918,0.036081,0.014795,0.000827,0.001332,0.000133,0.000338,...,0.000562,0.000671,0.000018,0.000051,0.000009,0.000008,0.002397,0.000015,0.000012,0.000047
std,1177.831997,28943.930805,0.004226,0.005064,0.004679,0.002147,0.000664,0.000576,0.000112,0.000216,...,0.000229,0.000283,0.000025,0.000055,0.000014,0.000014,0.000889,0.000022,0.000045,0.000054
min,460.000000,1575.000000,0.015659,0.010931,0.023693,0.004394,0.000000,0.000295,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000161,0.000000,0.000000,0.000000
25%,4767.250000,51384.750000,0.035457,0.036979,0.033086,0.013601,0.000435,0.001026,0.000055,0.000177,...,0.000400,0.000483,0.000000,0.000013,0.000000,0.000000,0.001794,0.000000,0.000000,0.000014
50%,5324.500000,64916.500000,0.038217,0.039824,0.035853,0.014822,0.000568,0.001195,0.000107,0.000308,...,0.000522,0.000624,0.000013,0.000036,0.000000,0.000000,0.002320,0.000010,0.000000,0.000035
75%,5893.500000,80526.000000,0.040611,0.043145,0.038983,0.016067,0.000910,0.001456,0.000180,0.000448,...,0.000681,0.000833,0.000024,0.000071,0.000015,0.000013,0.002906,0.000021,0.000016,0.000064
max,10469.000000,271760.000000,0.051376,0.056662,0.055229,0.021395,0.004063,0.004683,0.000837,0.001453,...,0.001421,0.001851,0.000172,0.000408,0.000109,0.000094,0.006602,0.000169,0.000953,0.000667


## Download 2021 CCM Data

We download CCM data from GitHub to incorporate additional firm information into our analysis.

In [14]:
# Load CRSP dataframe
crsp_2021_url = 'https://github.com/LeDataSciFi/data/raw/main/Firm%20Year%20Datasets%20(Compustat)/2021_ccm_cleaned.dta'
crsp_2021 = pd.read_stata(crsp_2021_url)

In [15]:
sp500_orig.head()

,Symbol,Security,CIK,truth_path
0,MMM,3M,66740,MMM
1,AOS,A. O. Smith,91142,AOS
2,ABT,Abbott,1800,ABT
3,ABBV,AbbVie,1551152,ABBV
4,ACN,Accenture,1467373,ACN


## Merge Sentiment Scores, Returns, and 2021 CCM Data

Now, we merge our sets together and save the final dataframe for analysis.

In [16]:
sp500_final = final_rets.merge(sp500_sents, on=['Symbol', 'Security', 'CIK'], validate='1:1', how='outer')
sp500_final = sp500_final.merge(crsp_2021.rename(columns={'tic':'Symbol'}),
        on='Symbol',
        how='left',
        validate='1:1')
sp500_final.to_csv(final_save_path, index=False)
sp500_final

,Symbol,Security,CIK,filing_date,ret_t-t2,ret_t3-t10,truth_path,unique_word_count,word_count,bhr_negative,...,mb,prof_a,ppe_a,cash_a,xrd_a,dltt_a,invopps_FG09,sales_g,dv_a,short_debt
0,ADBE,Adobe Inc.,796343,2022-01-21,-0.015915,0.004955,ADBE,5488.0,56254.0,0.038131,...,12.136953,0.233582,0.077677,0.212841,0.093242,0.167982,11.908058,0.226686,0.000000,0.020758
1,LMT,Lockheed Martin,936468,2022-01-25,-0.034930,0.057001,LMT,5251.0,75996.0,0.041739,...,2.677847,0.205630,0.174886,0.070843,0.029485,0.251017,2.188377,0.025169,0.057791,0.025340
2,SLB,Schlumberger,87347,2022-01-26,0.066684,0.970083,SLB,3735.0,35258.0,0.038289,...,1.651087,0.117680,0.214931,0.075619,0.013346,0.335188,1.369396,-0.028473,0.016839,0.072956
3,URI,United Rentals,1067701,2022-01-26,0.072747,0.202582,URI,4654.0,60311.0,0.040142,...,1.890685,0.212399,0.589198,0.007096,0.000000,0.463237,1.597794,0.139039,0.000000,0.105443
4,INTC,Intel,50863,2022-01-27,0.165922,0.660988,INTC,6519.0,68626.0,0.038775,...,1.678206,0.201145,0.378811,0.168717,0.090199,0.200735,1.454506,0.014859,0.033514,0.123678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,BF.B,Brown–Forman,14693,NaN,NaN,NaN,0000014693,4815.0,44204.0,0.036897,...,5.639590,0.210576,0.148909,0.137141,0.000000,0.325279,5.401004,0.136377,0.130394,0.115614
499,FRC,First Republic Bank,1132979,NaN,NaN,NaN,0001132979,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,GEHC,GE HealthCare,1932393,NaN,NaN,NaN,0001932393,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,SBNY,Signature Bank,1288784,NaN,NaN,NaN,0001288784,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
